# Submodule 3 Tutorial 2: Advanced Object-Oriented Programming in Python
---------------------------------------------------------------

### Overview
Creating a class object is a foundational task in OOP. In this tutorial, we will expand the way a class works to include useful checks on the attributes and define new methods. This module is not *necessary* for using classes, but will allow you to ease the use of bioinformatics datasets and avoid confusing behavior in your Python programs

### Learning Objectives

*After this tutorial, you should be able to*

- Recognize the difference between variable behavior within objects
- Use getters and setters
- Decorate attributes
- Create nested functions
- Use functions as parameters

### Prerequisites

Introductory level python
Introduction to OOP (Tutorial 1)

### Getting Started

Run the next code box to install needed libraries.

In [ ]:
%pip install jupyterquiz
from jupyterquiz import display_quiz
from datetime import date,datetime

## Instance Variables

Now that we've worked with objects, let's take another look at variables and how they behave inside classes. Just like in bioinformatics, where data is stored at different levels (e.g., genome-wide vs. within a single gene), Python uses different types of variables depending on where and how they are used.

1️⃣ Global Variables – Always Available
<br>
🔹 These exist outside of any class or function and stay in memory for the entire session.
<br>
🔹 In bioinformatics, this could be a global setting for a sequencing run or a common reference genome used throughout an analysis.

2️⃣ Local Variables – Temporary & Limited
<br>
🔹 These are created inside a method and only exist while the method is running.
<br>
🔹 In bioinformatics, this could be a temporary list of nucleotide changes when analyzing a mutation.

3️⃣ Class Variables – Shared by All Objects
<br>
🔹 Declared inside the class but outside any method.
<br>🔹 Shared by all objects of the class—meaning if one object changes it, all objects see the update.
<br>🔹 In bioinformatics, this could be a global mutation database that all Mutation objects use.

4️⃣ Instance Variables – Unique to Each Object
<br>🔹 Created inside the constructor (__init__) using self.
<br>🔹 Each object gets its own copy—changes to one object do not affect others.
<br>🔹 In bioinformatics, this could be an individual patient's genetic mutation record.

<br>
In the next box, you can see an example of the fact that "translated codon" is not a global variable, so it cannot be accessed outside of the class, as well as examples of other variable types.

In [ ]:
# Global variable: Exists outside any class or method and can be accessed anywhere
global_genetic_code = {"ATG": "Methionine", "TAA": "Stop", "TAG": "Stop", "TGA": "Stop"}

class DNAAnalyzer:
    # Class attribute: Shared across all instances of this class
    species_name = "Unknown Species"

    def __init__(self, sequence):
        # Instance attribute: Unique to each object created from this class
        self.instance_sequence = sequence

    def find_start_codon(self):
        # Method-local variables: Exist only within this method during execution
        local_start_codon = self.instance_sequence[:3]
        translated_codon = global_genetic_code.get(local_start_codon, "No start codon found")

        return translated_codon

# Example usage:
sample = DNAAnalyzer("ATGCGTACG")  # Create an instance with a DNA sequence

# Accessing different variable types
print(f"Global variable (genetic code dictionary): {global_genetic_code}")  # ✅ Accessible
print(f"Class attribute (shared across instances): {DNAAnalyzer.species_name}")  # ✅ Accessible
print(f"Instance attribute (unique per object): {sample.instance_sequence}")  # ✅ Accessible
print(f"Method-local variable output: {sample.find_start_codon()}")  # ✅ Accessible via method

# 🚨 Attempting to access a method-local variable directly (WILL CAUSE AN ERROR but won't crash the code box because we just "tried")
try:
    print(f"Trying to access 'translated_codon' directly: {sample.translated_codon}")  # ❌ Not possible
except AttributeError:
    print("Error: 'translated_codon' is a method-local variable and cannot be accessed outside the method!")


### Why Is It Important to Understand Variable Access in Python?
Understanding how variables are accessed in Python is crucial because it determines how data is stored, modified, and shared across different parts of a program. In bioinformatics, just like in programming, data security, scope, and organization matter when handling sensitive or large datasets.

Here are just two examples of why you need to understand variable access:

1️⃣ <u>Avoiding Unintended Data Changes</u>
<br>If a class variable (shared by all objects) is mistakenly used instead of an instance variable (unique to each object), changes to one object can affect all objects, leading to unexpected bugs.
<br>
🔹 Example: Incorrect Use of Class Variable (Bug!)

In [ ]:
class Patient:
    mutations = []  # Shared class variable (BAD practice!)

    def __init__(self, patient_id, mutation):
        self.patient_id = patient_id
        self.mutations.append(mutation)  # This modifies the shared list

# Creating two patient objects
p1 = Patient("P001", "BRCA1 A→G")
p2 = Patient("P002", "TP53 C→T")

print(p1.mutations)  # Both patients now share the same mutation list!
print(p2.mutations)  # Output: ['BRCA1 A→G', 'TP53 C→T'] (Oops!)


📌 Problem: The mutations list was meant to be unique for each patient, but instead, all patients share the same list because it's a class variable.
<br>
✅ Fix: Use an instance variable (self.mutations) inside __init__ instead.

2️⃣<u>Controlling Variable Scope and Security</u>
<br>In Python, variables can be public, protected, or private. In bioinformatics, patient data is sensitive, and it's important to prevent accidental changes to critical information.

🔹 Example: Protecting Patient Data with Private Variables

In [ ]:
class Patient:
    def __init__(self, patient_id, diagnosis):
        self.patient_id = patient_id  # Public attribute
        self.__diagnosis = diagnosis  # Private attribute (Cannot be accessed directly-- it has 2 underscores)

    def get_diagnosis(self):
        """Allows controlled access to the diagnosis."""
        return self.__diagnosis

# Creating a patient object
p = Patient("P123", "Breast Cancer")
print(p.get_diagnosis())  # Allowed ✅ 
#print(p.diagnosis) # ❌ AttributeError: While it was entered as a diagnosis, note that it was renamed in the __init__
#print(p.__diagnosis)  # ❌ AttributeError: Private variable cannot be accessed directly!

#### Why is this important?
<ul>
<li>Prevents accidental modifications of sensitive data.
<li>Encourages using getter methods (get_diagnosis()) for controlled access.
</ul>


### Test your Understanding
Time for a quiz to see how well you understand variables and how to control them in Python.

Question 2 refers to this code: (feel free to create the class in a code box to evaluate)
class Example:
    value = 10 

    def __init__(self):
        self.value = 20 
        self.__mutation ="rs45677"

obj=Example()

In [ ]:
from jupyterquiz import display_quiz
variable_vocab="PythonQuizQuestions/var_qz.json"
display_quiz(variable_vocab)

## Attributes
Attributes act as a window into a class’s internal data. They allow objects to store and manage information, such as a patient's diagnosis, genetic data, or calculated age. In some programming languages, attributes are called properties, serving as a way to interact with the data inside a class.
<br>
Typically, when we declare variables inside the __init__ constructor, we are creating instance attributes—these belong to each individual object and are referenced using an underscore (_) to indicate they should not be accessed directly. However, not all attributes need to be set manually; some, like age, can be calculated dynamically based on stored information.

In the following example, we will create some attributes manually (using __init__) and others with a "decorator"

Instead of storing a patient’s age as a fixed value (which would become outdated), we store their date of birth (_dob) and use a calculated property to determine their age whenever it is needed. This ensures that age is always accurate without requiring manual updates. 

We introduce here a new symbol & vocabulary: @property   

The **@property** decorator in Python allows you to <u>define a method that behaves like an attribute.</u> This means you can access it without parentheses, just like a normal variable, but it calculates or retrieves data dynamically.

In our case, we used @property to calculate age based on date of birth (_dob). This way, we don’t have to store or manually update age—it updates automatically every time we access it.


In [ ]:
from datetime import date,datetime

class Patient:
    """Represents a patient with calculated age and protected attributes."""
    
    #Class Attributes, characteristics of ALL patients
    hospital="Community General"
    informed_consent="Y"
    study_ID = "F-2720"
    
    def __init__(self, name, dob, diagnosis):
        """Initialize patient with name, date of birth, and diagnosis."""
        self.name = name
        self._dob = dob  # Store date of birth as a protected attribute
        self._diagnosis = diagnosis  # Store diagnosis as a protected attribute

    @property
    def age(self):
        """Calculates the patient's age dynamically from date of birth."""
        today = date.today()
        dob_date = datetime.strptime(self._dob, "%Y-%m-%d") #this tool knows how to turn date strings into a Date object
        return today.year - dob_date.year - ((today.month, today.day) < (dob_date.month, dob_date.day))

    def get_diagnosis(self):
        """Allows controlled access to the diagnosis."""
        return self._diagnosis

# Creating an instance of Patient
p = Patient("John Doe", "1980-05-12", "Congestive Heart Failure")

# Accessing calculated age
print(f"{p.name} is {p.age} years old.")  # ✅ Output depends on today's date

# Accessing diagnosis safely
print("Diagnosis:", p.get_diagnosis())  # ✅ Output: Congestive Heart Failure

# Access the class attribute
print("Study ID:",p.study_ID)

You can see above that you can access the instance attributes using the “dot” syntax. Try to use the same technique for the diagnosis nor the date of birth (dob) which we made protected attributes.

In [ ]:
p.name
#p.age
#p.dob
#p.diagnosis

We *can* access those attributes, however. They are only slightly protected:

In [ ]:
p._dob
#p._diagnosis

<div class="alert alert-block alert-info"> <b>Try this:</b> Edit the code above to take date of diagnosis date as an attribute of Patient. Then, calculate the years since diagnosis as a decorator before printing it along with the diagnosis</a> </div>

In [ ]:
#enter your code here

In [ ]:
#example code to make that calculation
    def __init__(self, name, dob, dod, diagnosis):
        """Initialize patient with name, date of birth, date of diagnosis (dod) and diagnosis."""
        self.name = name
        self._dob = dob  # Store date of birth as a protected attribute
        self._dod = dod #Store date of diagnosis as a protected attribute 
        self._diagnosis = diagnosis  # Store diagnosis as a protected attribute

    @property
    def years_since_diagnosis(self):
        """Calculates the time since diagnosis dynamically from diagnosis."""
        today = date.today()
        date_of_diag = datetime.strptime(self._dod, "%Y-%m-%d") 
        return today.year - date_of_diag.year - ((today.month, today.day) < (date_of_diag.month, date_of_diag.day))

p2 = Patient("John Mendez", "1981-05-12", "2023-07-14", "Cancer") #test patient

## Error Checking in the Constructor

Our code works well for calculating age, but look how it handles dates in the future (probably a typo in the data entry):


In [ ]:
p1=Patient("p001","2090-12-02", "Congestive Heart Failure")
p1.age

If we were to get the average age of patients, including negative values, that could cause substantial problems!  

We will examine two types of **error checking**: in the constructor and in a decorator.

Error checking in the constructor (__init__) ensures that invalid data is caught immediately when an object is created. 

Let's examine how to validate that the <u>diagnosis</u> is from an allowed list.

You can see that **before** we store the provided diagnosis value in self.diagnosis, we first eliminate issues with capitalization (the interpretation of strings is VERY literal, as you remember, such that diabetes is not Diabetes).

We also check that the provided diagnosis is in our short list of options. **If** it is, then we can store it as self._diagnosis


In [ ]:
from datetime import date,datetime

class Patient:
    """Represents a patient with calculated age and protected attributes."""
    
    #Class Attributes, characteristics of ALL patients
    hospital="Community General"
    informed_consent="Y"
    study_ID = "F-2720"
   
    # Define allowed diagnoses
    ALLOWED_DIAGNOSES = {"healthy", "diabetes", "cancer"}
        
    def __init__(self, name, dob, diagnosis):
        """Initialize patient with name, date of birth, and diagnosis."""
        self.name = name
        self._dob = dob  # Store date of birth as a protected attribute
 
        # ✅ Validate diagnosis (must be in allowed list)
        if diagnosis.lower() not in self.ALLOWED_DIAGNOSES:
            raise ValueError(f"Error: '{diagnosis}' is not a valid diagnosis. Choose from {self.ALLOWED_DIAGNOSES}.")
        self._diagnosis = diagnosis  # Store valid diagnosis
    

    @property
    def age(self):
        """Calculates the patient's age dynamically from date of birth."""
        today = date.today()
        dob_date = datetime.strptime(self._dob, "%Y-%m-%d") #this tool knows how to turn date strings into a Date object
        return today.year - dob_date.year - ((today.month, today.day) < (dob_date.month, dob_date.day))

    def get_diagnosis(self):
        """Allows controlled access to the diagnosis."""
        return self._diagnosis

# Creating an instance of Patient
#p = Patient("John Doe", "1980-05-12", "Congestive Heart Failure")
p1= Patient("p001","1990-12-02", "Cancer")
p1.get_diagnosis()


<div class="alert alert-block alert-info"> <b>Try this:</b> Edit the code above to take sex as an attribute of Patient. Limit the options to your own definition in the constructor. ADVANCED: convert a range of options to common values such as male or m or Male--> M </a></div>

In [ ]:
#suggested code for the Try this
from datetime import date,datetime

class Patient:
    """Represents a patient with calculated age and protected attributes."""
    
    #Class Attributes, characteristics of ALL patients
    hospital="Community General"
    informed_consent="Y"
    study_ID = "F-2720"
   
    # Define allowed diagnoses
    ALLOWED_DIAGNOSES = {"healthy", "diabetes", "cancer"}
    sexes= {"MALE","FEMALE","OTHER"}
    sex_abbrev={"M","F","O"}
        
    def __init__(self, name, dob, sex, diagnosis):
        """Initialize patient with name, date of birth, and diagnosis."""
        self.name = name
        self._dob = dob  # Store date of birth as a protected attribute
 
        # ✅ Validate diagnosis (must be in allowed list)
        if diagnosis.lower() not in self.ALLOWED_DIAGNOSES:
            raise ValueError(f"Error: '{diagnosis}' is not a valid diagnosis. Choose from {self.ALLOWED_DIAGNOSES}.")
        self._diagnosis = diagnosis  # Store valid diagnosis

        if sex.upper() not in self.sexes:
            if sex not in self.sex_abbrev:
                raise ValueError(f"Error: '{sex}' is not a valid designation for this study. Choose from {self.sexes}.")
        self.sex=sex[0].upper()

    @property
    def age(self):
        """Calculates the patient's age dynamically from date of birth."""
        today = date.today()
        dob_date = datetime.strptime(self._dob, "%Y-%m-%d") #this tool knows how to turn date strings into a Date object
        return today.year - dob_date.year - ((today.month, today.day) < (dob_date.month, dob_date.day))

    def get_diagnosis(self):
        """Allows controlled access to the diagnosis."""
        return self._diagnosis

p3=Patient("jack sprat","1970-08-21","female","Cancer")
p3.sex

We should build into our class some error checking for the dates. In this case, we know that the birth date should be in the future (in Python that means today > date of birth). We will add this as an attribute.

A technique we can use is a *try block*, which lets you test some code for errors, without crashing the code if an error is encountered. Then,

- The *except block* lets you handle the error.
- The *else block* lets you execute code when there is no error.

See how it works in the date check. Look at the  def _validate_dob(self, dob). 

The datetime tool will convert a text string, in a provided format (e.g., "%Y-%m-%d") into a variable of the date class. As before, we'll call on today's date to make sure that the date of birth is in the past (but you could check to make sure the person is within an age range or over 18 yrs old, for example)

In [ ]:
from datetime import datetime

class Patient:
    """Represents a patient with error-checked attributes."""

    # Define allowed diagnoses
    ALLOWED_DIAGNOSES = {"healthy", "diabetes", "cancer", "congestive heart failure"}
    
    def __init__(self, name, dob, diagnosis):
        """Initialize patient with name, date of birth, and diagnosis."""
        self.name = name # Always valid
        self._dob = self._validate_dob(dob)  # Validate and store DOB
     
        # ✅ Validate diagnosis (must be in allowed list or NA)
        if diagnosis.lower() not in self.ALLOWED_DIAGNOSES:
            self.diagnosis = "NA"
        else:
            self.diagnosis = diagnosis  # Store valid diagnosis

    def _validate_dob(self, dob):
        """Private method to validate DOB format and ensure it's not in the future."""
        today = datetime.today()
        try:
            # Parse the date to check format
            dob_date = datetime.strptime(dob, "%Y-%m-%d")
        except ValueError:
            return "❌ Error: Invalid format! Use YYYY-MM-DD."
             # Check if it's before today's date      
        if dob_date >= today:
            return "❌ Error: Date of birth cannot be in the future."
        
        return dob
    
    @property
    def age(self):
        """Calculates the patient's age dynamically from date of birth."""
        today = datetime.today()
        try:
            dob_date = datetime.strptime(dob, "%Y-%m-%d")
        except ValueError:
            return "DOB format problem or in the future"
        return today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))

p1=Patient("p001","2000-12-02", "COPD")
p1._diagnosis

<div class="alert alert-block alert-info"> <b>Try these:</b> (1) Edit the code above to make diagnosis a private variable by adding a getter-- to get_diagnosis as above  (2) </a> </div>

#### Test your knowledge
Just a quick check!

In [ ]:
from jupyterquiz import display_quiz
var2="PythonQuizQuestions/var2.json"
display_quiz(var2)

## Nested Functions

Let's go back to functions for a minute. A nested function is a function defined inside another function. The inner function is only accessible within the outer function, meaning it cannot be called independently. Nested functions help organize code, encapsulate logic, and avoid unnecessary global variables, for example.

In [ ]:
# Example of a nested function
def transcribe_dna(dna_sequence):
    """Outer function that processes a DNA sequence for transcription."""

    def convert_to_mrna(seq):
        """Inner function that replaces 'T' with 'U' to create mRNA."""
        return seq.replace("T", "U")

    # Display original DNA sequence
    print("Original DNA:", dna_sequence)

    # Convert and display transcribed mRNA
    mrna_sequence = convert_to_mrna(dna_sequence)
    print("Transcribed mRNA:", mrna_sequence)

# Example Usage
transcribe_dna("ATGCTTGA")


There are several benefits from creating and using nested functions
<br>✅ Encapsulated logic – The inner function exists only within the outer function.
<br>✅ Modular & readable – Helps break down complex tasks into smaller parts.
<br>✅ Avoids global clutter – Variables inside the inner function don’t affect the rest of the program.
<br>✅ Provides closures - they can maintain state after the outermost function has finished executing.


### Test your knowledge
<div class="alert alert-block alert-info"> <b>Try this:</b> Write a Python function called nucleotide_analysis(dna_sequence) that:
<br> - Defines a nested function to calculate the frequency of each nucleotide (A, T, C, G).
<br> - Uses the nested function to generate and return a dictionary with nucleotide counts.
<br> - Prints the results as a formatted table.
</a> </div>
<br>
<br><b>What the Function Should Do</b>
<br>✔ Accepts a DNA sequence (e.g., "ATGCGATCGT"):  nucleotide_analysis("ATGCGATCGT")
<br>✔ Uses a nested function to count nucleotides
<br>✔ Returns and prints a table of nucleotide frequencies
<br>
<i>Solution at the end of this tutorial</i>

<b>Suggestions</b>
<ul>
<li>The outer function takes in a DNA sequence.
<li>The inner function (nested function) creates and fills a dictionary {A: count, T: count, C: count, G: count}.
<li>Use .count(nucleotide) to count each nucleotide.
<li>Format the table using a loop.
</ul>


In [ ]:
#Try your code here

## Other Python Programming Objects

### Closures
A *closure* is a function that defines a nested function and returns it, allowing the inner function to be used outside its original scope while retaining access to variables from the outer function.
The term closure comes from the idea that a function "closes over" variables from its outer scope, keeping them alive even after the outer function has finished executing.

Closures improve code flexibility by enabling function factories, decorators, and callback functions. Here’s how they are commonly used:

- Closures power decorators – Decorators modify the behavior of functions without changing their code.
- Closures generate callback functions – Common in event-driven programming (e.g., GUI applications), where functions respond dynamically to user actions.
- Closures act as function "factories" – They create multiple functions from a template, each customized with specific behavior.

Here is a simple closure (a "function factory")

In [ ]:
# Here's a simple example of a function factory
# It's a simple "rooter" that takes a number n and creates a function
# That will generate the nth root of an input.
# Hence, we can create a whole slew of root functions on the fly.
def root_factory(root_value):
    def root_function(number):
        return number ** (1/root_value) # remember that ** makes the next value an exponent
    return root_function

# Create specific functions using the factory, square root, cube root, fourth root
square_root = root_factory(2)
cube_root = root_factory(3)
fourth_root = root_factory(4)

# Test the functions
print(square_root(25))
print(cube_root(27))
print(fourth_root(256))


The key benefit to the above root_factory() is that it technically creates multiple functions without rewriting the logic, making it efficient and reusable.
<br>
Below, we demonstrate creating a custom sequence filter using a closure
<br>
Closures are useful in bioinformatics when we need to generate filtering functions dynamically. For example, we may want to filter DNA sequences based on length, GC content, or the presence of specific motifs. Instead of writing multiple filter functions manually, we can use a closure to generate them on demand.
<br>
<br>
📌 Problem: We need a function that can generate <b>different</b> filtering functions based on criteria such as:
<ul>
<li>Minimum sequence length
<li>Minimum GC content
</ul>

In [ ]:
def sequence_filter_factory(min_length=0, min_gc_content=0):
    """Creates a function that filters DNA sequences based on length and GC content.
    The user will supply numbers in place of zeroes"""
    
    def filter_sequence(dna_sequence):
        """Inner function that applies filtering criteria."""
        dna_sequence = dna_sequence.upper()
        # Check sequence length
        if len(dna_sequence) < min_length:
            return False
        
        # Calculate GC content
        gc_content = (dna_sequence.count("G") + dna_sequence.count("C")) / len(dna_sequence) * 100
        if gc_content < min_gc_content:
            return False
        
        return True  # Sequence meets all criteria, since False was not returned by any inner function
    
    return filter_sequence  # Return the inner function (closure)

# ✅ Example Usage

# Create a filter for sequences of at least 10 bases and 50% GC content
filter_10bp_50gc = sequence_filter_factory(min_length=10, min_gc_content=50)

# Test sequences
sequences = ["ATGC", "GCGCGCGC", "ATGCGTACGTGCA", "ATATATAT", "CGCGCGAUGAAA"]

# Apply filter as part of a short iterator over sequences
filtered_sequences = [seq for seq in sequences if filter_10bp_50gc(seq)]  #remember that any variable name works in place of seq
print(filtered_sequences)  # ✅ Output for 10 & 50: ['ATGCGTACGTGCA', 'CGCGCGAAAA']


Notice that the inner filters are all pass-filters: if a criteria is NOT met, then False is returned and the sequence will not be included in the final output.

Perhaps you could imagine other features you could add to this filter factory? 

- Only ones with a start codon?  sequence_filter_factory(min_length=0, min_gc_content=0, start_codon=False) (code provided below)
- only ones with the AUG at a particular location in the sequence, provided by the user?

You can see the efficiency of writing closures!

In [ ]:
#Example code here
def sequence_filter_factory(min_length=0, min_gc_content=0, start_codon=False):
    """Creates a function that filters DNA sequences based on provided length, GC content, and if an AUG is present.
    The user will supply numbers in place of zeroes"""
    
    def filter_sequence(dna_sequence):
        dna_sequence = dna_sequence.upper()
        """Inner function that applies filtering criteria."""
        # Check sequence length
        if len(dna_sequence) < min_length:
            return False
        
        # Calculate GC content
        gc_content = (dna_sequence.count("G") + dna_sequence.count("C")) / len(dna_sequence) * 100
        if gc_content < min_gc_content:
            return False

        # Is there a start codon, if requested
        if start_codon:
            count_AUG = dna_sequence.count("AUG")
            if count_AUG == 0:
                return False
        
        return True  # Sequence meets all criteria, since all inner "questions" were answered TRUE
    
    return filter_sequence  # Return the inner function (closure)

# ✅ Example Usage

# Create a filter for sequences of at least 10 bases and 50% GC content, with a start codon
filter_10bp_50gc_start = sequence_filter_factory(min_length=10, min_gc_content=50, start_codon=True)

# Test sequences
sequences = ["CGCGCGAUGAAA","ATGC", "GCGCGCGC", "ATGCGTACGTGCA", "ATATATAT", "CGCGCGAUGAAA"]

# Apply filter as part of a short iterator over sequences
filtered_sequences = [seq for seq in sequences if filter_10bp_50gc_start(seq)]  #remember that any variable name works in place of seq
print(filtered_sequences)  # ✅ Output for 10 & 50: ['ATGCGTACGTGCA', 'CGCGCGAAAA']


### Test your understanding

Run the next code box for a quiz

In [ ]:
from jupyterquiz import displayquiz


## Functions as Parameters

Because functions are an object like any other variable, Python allows us to pass them just as we would any other variable.
- We have already seen functions used as paramters in closures, but it is even simpler than that. You can just create functions and pass them to any other function:

In [ ]:
# Create a couple of simple functions.
def myUpper(mystr):
    return mystr.upper()

def myLower(mystr):
    return mystr.lower()

# New create a function that takes as a parameter another function.
# This is a pretty common practice in data science. We often want to create data-dependent functions during the feature engineering process
# to handle data from independent sources or data that varies widely under certain conditions.
# An example would be sales data of say, beachware, in summer and winter seasons.

def worldVirus(myfunc):
    virus = myfunc("Covid-19")
    print(virus)

# Test
worldVirus(myUpper)
worldVirus(myLower)


## Conclusion

In this tutorial, you have learned:
- Some OOP vocabulary
- Defined a new class
- Added attributes and decorators
- Created nested functions and Closures

Obviously, there are many other ways that you can use *and reuse* objects in Python to make your coding efficient. After this short introduction, though, it is hoped that you can read other code to interpret how a class or other objects are being used to simplify the hard work of coding for bioinformatics.

You are ready for the project 


## Clean up
Remember to shut down your compute instance when you are done for the day to avoid unnecessary charges. You can do this by stopping the notebook instance from the Cloud console.

In [ ]:
#Solution to nested function
def nucleotide_analysis(dna_sequence):
    """Outer function that processes a DNA sequence and calculates nucleotide frequencies."""

    def count_nucleotides(seq):
        """Inner function that returns a dictionary of nucleotide counts."""
        counts = {"A": 0, "T": 0, "C": 0, "G": 0}  # Initialize dictionary
        for nucleotide in seq:
            if nucleotide in counts:  # Ensure only valid nucleotides are counted
                counts[nucleotide] += 1
        return counts

    # Get nucleotide counts from the inner function
    nucleotide_counts = count_nucleotides(dna_sequence)

    # Print results as a table
    print("\nNucleotide Counts:")
    for nucleotide, count in nucleotide_counts.items():
        print(f"{nucleotide}: {count}")

    return nucleotide_counts  # Return dictionary for further use if needed

# Example Usage
nucleotide_analysis("ATGCGATCGT")
